## find_data_splits_dir_dev.ipynb

Added on: Thursday April 21st, 2022  
Adapted by: Jacob Alexander Rose  

In [6]:
256+128

384

In [5]:
f"{3*1.746e-3/2:e}"

'2.619000e-03'

In [1]:
%load_ext autoreload
%autoreload 2


import lmdb

from imutils.ml.utils.etl_utils import ETL
from omegaconf import DictConfig, OmegaConf
import os
from rich import print as pp
import hydra


import numpy as np
import pyarrow as pa
pa.__version__

from typing import *
import inspect
from tqdm.auto import tqdm
import pandas as pd
from pathlib import Path
from imutils.ml.utils.template_utils import get_logger
import logging

logger = get_logger(name=__name__, level=logging.INFO)

Importing imutils


In [2]:
import imutils

HERBARIUM_ROOT_DEFAULT = os.environ.get("HERBARIUM_ROOT_DEFAULT")
CATALOG_DIR = os.environ.get("CATALOG_DIR")
SPLITS_DIR = os.environ.get("SPLITS_DIR")

In [62]:
def find_data_splits_dir(source_dir: str,
                         stratify_col: str="scientificName",
                         train_size: float=None,
                         splits: Tuple[float]=(0.5, 0.2, 0.3)) -> Path:
    """
    Given a base path of `source_dir`, construct the correct data split dir path using chosen train_size.
    
    [TODO] Formalize the test cases for this function
    """

    source_dir = Path(source_dir)
    stratify_str = f"stratify-{stratify_col}"
    if "splits" not in str(source_dir):
        source_dir = source_dir / "splits"
    # os.makedirs(source_dir, exist_ok=True)

    if isinstance(train_size, float) and splits is None:    
        splits_subdir_str = f"train_size-{train_size:.1f}"
    else:
        splits_subdir_str = f"splits-({splits[0]:.1f},{splits[1]:.1f},{splits[2]:.1f})"
        
    splits_subdir_str_base = splits_subdir_str.split("-")[0] + "-"
        

    if splits_subdir_str in str(source_dir):
        print(0)
        return source_dir
    elif splits_subdir_str_base in source_dir.name:
        print(1)
        return source_dir.parent / splits_subdir_str
    if stratify_str in str(source_dir.name):
        print(2)
        return source_dir / splits_subdir_str
    elif "stratify" in str(source_dir.name):
        print(3)
        return source_dir.parent / stratify_dir / splits_subdir_str
    else:
        print(4)
        return source_dir / stratify_str / splits_subdir_str

    
    
    
#     if isinstance(train_size, float) and splits is None:    
#         train_split_str = f"train_size-{train_size:.1f}"
#         if train_split_str in str(source_dir):
#             return source_dir
#         elif "train" in source_dir.name:
#             return source_dir.parent / train_split_str
#         if stratify_str in str(source_dir):
#             return source_dir / train_split_str
#         else:
#             return source_dir / stratify_str / train_split_str
    
#     else:
#         splits_subdir_str = f"splits=({splits[0]:.1f},{splits[1]:.1f},{splits[2]:.1f})"

#         if splits_subdir_str in str(source_dir):
#             return source_dir
#         elif "splits=" in source_dir.name:
#             return source_dir.parent / splits_subdir_str
#         if stratify_str in str(source_dir):
#             return source_dir / splits_subdir_str
#         else:
#             return source_dir / stratify_str / splits_subdir_str


## Testing type I. train_size-based splits

In [63]:
source_dir = SPLITS_DIR
stratify_col="scientificName"
train_size=0.7
splits=None

In [64]:
found_split_dir = find_data_splits_dir(source_dir=source_dir,
                                       stratify_col=stratify_col,
                                       train_size=train_size,
                                       splits=splits)
print(f"Input: {source_dir}")
print(f"Found: {found_split_dir}")

1
Input: /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits/stratify-scientificName/train_size-0.8
Found: /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits/stratify-scientificName/train_size-0.7


In [65]:
input_dir = Path(source_dir).parent

found_split_dir = find_data_splits_dir(source_dir=source_dir,
                                       stratify_col=stratify_col,
                                       train_size=train_size,
                                       splits=splits)
print(f"Input: {input_dir}")
print(f"Found: {found_split_dir}")

1
Input: /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits/stratify-scientificName
Found: /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits/stratify-scientificName/train_size-0.7


In [66]:
input_dir = Path(source_dir).parent.parent

found_split_dir = find_data_splits_dir(source_dir=source_dir,
                                       stratify_col=stratify_col,
                                       train_size=train_size,
                                       splits=splits)
print(f"Input: {input_dir}")
print(f"Found: {found_split_dir}")

1
Input: /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits
Found: /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits/stratify-scientificName/train_size-0.7


In [67]:
input_dir = Path(source_dir).parent.parent.parent

found_split_dir = find_data_splits_dir(source_dir=source_dir,
                                       stratify_col=stratify_col,
                                       train_size=train_size,
                                       splits=splits)
print(f"Input: {input_dir}")
print(f"Found: {found_split_dir}")

1
Input: /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs
Found: /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits/stratify-scientificName/train_size-0.7


## Testing type II. tuple-based splits

In [73]:
source_dir = " /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits/stratify-scientificName/splits-(0.5,0.3,0.2)"
stratify_col="scientificName"
train_size=None
splits=(0.5,0.3,0.2)

In [74]:
found_split_dir = find_data_splits_dir(source_dir=source_dir,
                                       stratify_col=stratify_col,
                                       train_size=train_size,
                                       splits=splits)
print(f"Input: {source_dir}")
print(f"Found: {found_split_dir}")

0
Input:  /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits/stratify-scientificName/splits-(0.5,0.3,0.2)
Found:  /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits/stratify-scientificName/splits-(0.5,0.3,0.2)


In [75]:
input_dir = Path(source_dir).parent

found_split_dir = find_data_splits_dir(source_dir=source_dir,
                                       stratify_col=stratify_col,
                                       train_size=train_size,
                                       splits=splits)
print(f"Input: {input_dir}")
print(f"Found: {found_split_dir}")

0
Input:  /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits/stratify-scientificName
Found:  /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits/stratify-scientificName/splits-(0.5,0.3,0.2)


In [76]:
input_dir = Path(source_dir).parent.parent

found_split_dir = find_data_splits_dir(source_dir=source_dir,
                                       stratify_col=stratify_col,
                                       train_size=train_size,
                                       splits=splits)
print(f"Input: {input_dir}")
print(f"Found: {found_split_dir}")

0
Input:  /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits
Found:  /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits/stratify-scientificName/splits-(0.5,0.3,0.2)


In [77]:
input_dir = Path(source_dir).parent.parent.parent

found_split_dir = find_data_splits_dir(source_dir=source_dir,
                                       stratify_col=stratify_col,
                                       train_size=train_size,
                                       splits=splits)
print(f"Input: {input_dir}")
print(f"Found: {found_split_dir}")

0
Input:  /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs
Found:  /media/data/home/jrose3/data_cache/herbarium-2022-fgvc9_resize-512/catalogs/splits/stratify-scientificName/splits-(0.5,0.3,0.2)
